In [ ]:
from scipy.io import wavfile
import numpy as np
import scipy
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
# from IPython.display import Audio 
# from vscode_audio import Audio
import IPython.display
import librosa
import librosa.display
import re

def myAudio(audio: np.ndarray, sr: int):
    """
    Use instead of IPython.display.Audio as a workaround for VS Code.
    `audio` is an array with shape (channels, samples) or just (samples,) for mono.
    """

    if np.ndim(audio) == 1:
        channels = [audio.tolist()]
    else:
        channels = audio.tolist()

    return IPython.display.HTML("""
        <script>
            if (!window.audioContext) {
                window.audioContext = new AudioContext();
                window.playAudio = function(audioChannels, sr) {
                    const buffer = audioContext.createBuffer(audioChannels.length, audioChannels[0].length, sr);
                    for (let [channel, data] of audioChannels.entries()) {
                        buffer.copyToChannel(Float32Array.from(data), channel);
                    }
            
                    const source = audioContext.createBufferSource();
                    source.buffer = buffer;
                    source.connect(audioContext.destination);
                    source.start();
                }
            }
        </script>
        <button onclick="playAudio(%s, %s)">Play</button>
    """ % (json.dumps(channels), sr))

In [ ]:
rate, data = wavfile.read('bass_electronic_018-022-100.wav')

In [ ]:
rate

In [ ]:
data

In [ ]:
# with open('..\examples.json') as what_file:
#     readit = json.load(what_file)

# train = pd.DataFrame.from_dict(readit).transpose().groupby(by='instrument_str', as_index=True)
# train.head()

In [ ]:
plt.figure()
data, rate = librosa.load('bass_synthetic_033-082-100.wav')
librosa.display.waveshow(data, sr=rate)
display(myAudio(data, rate))

In [ ]:
meta = pd.DataFrame(os.listdir()).set_axis(['wavfile'], axis=1)
cat = [None]*meta.size
for i in range(meta.size):
    cat[i] = re.split("_", re.sub("_", ":", meta.wavfile[i], 1))[0]
meta['label'] = cat

In [ ]:
meta

In [ ]:
choice = meta.label.unique()[:10]
choice

In [ ]:
train = pd.DataFrame()
for i in range(10):
    c = choice[i]
    train = pd.concat([train, meta.query("label == @c").sample(40)], ignore_index=True)
train

In [ ]:
features_df = pd.DataFrame()
X = [None]*train.wavfile.size
y = train.label

def scaledNNfeatures(filename):
    data, rate = librosa.load(filename)
    return np.mean(librosa.feature.mfcc(y=data, sr=rate, n_mfcc=40).T, axis=0)

for i in range(train.wavfile.size):
    X[i] = scaledNNfeatures(train.wavfile[i])
    X[i] = X[i].reshape(40)
    y[i] = train.label[i]

features_df['features'] = X
features_df['label'] = y

In [ ]:
X=np.array(X)

In [ ]:
features_df = features_df.to_numpy()

In [ ]:
features_df

## Label Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
#import pandas as pd
#import scipy
#import numpy
#from tensorflow.keras.utils import to_categorical
#from sklearn.preprocessing import LabelEncoder

# X = np.array(features_df.features)
#y = scipy.sparse.coo_matrix(to_categorical(LabelEncoder().fit_transform(features_df.label)))
#y = y.toarray().reshape(400,10)
# features_df.drop('label', axis=1, inplace=True)
# features_df

In [ ]:
# features_df.to_csv('../bool_features.csv')

## Train-Test Split

In [ ]:
#import tensorflow as tf
#tensor_features = tf.Tensor(features_df, dtype=int)
# tensor_features = tf.convert_to_tensor(features_df, np.ndarray)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
num_labels=y.shape[1]
num_labels

In [ ]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 3

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
old = pd.read_csv('mfcc_send.csv')
old

In [ ]:
list(range(40))

In [ ]:
new_data = pd.DataFrame(columns=list(range(40)))
# new_data.set_index(list(range(15000)))
# new_data[new_data.columns] = [[0]*15000]*40
for i in list(range(40)):
    new_data[new_data.columns[i]] = [0]*15000
new_data

In [ ]:
old.columns.size

In [ ]:
for i in range(new_data.columns.size):
    for j in range(old.features.size):
        new_data[new_data.columns[i]][j] = [float(s) for s in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", ' '+old.features[j][1:-1])][i]
new_data

In [ ]:
' '+old.features[0][1:-1]

In [ ]:
[float(s) for s in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", ' '+old.features[0][1:-1])]

In [ ]:
# re.split(' ', ' '+old.features[0][1:-2])[0]      # column number is [i] at the right end
# new_data['label'] = ([1]*3000 + [2]*3000 + [3]*3000 + [4]*3000 + [5]*3000)
# new_data.columns = list(range(41))
# new_data.drop(['label'], axis=1, inplace=True)
# new_data
new_data.to_csv('new_data.csv')